### load dataset 

In [3]:
from load_data import dataset

In [4]:
train, val, test = dataset.transform(max_length=2)

for t in train:
    sample = t["input_ids"]
    mask = t["attention_mask"]
    token = t["token_type_ids"]
    labels = t["labels"]
    print(sample.size())
    break 

torch.Size([16, 2])


### training step

+ saving local in `.onnc` model and `.pth` model_weights
+ upload s3 
+ add info to `MODEL_DB`

In [5]:
from networks.classifier_net import BertForIMDBClassification
from networks.load_config import config


net = BertForIMDBClassification(config)

INFO:networks.weights.load_weights_from_s3:loading s3 from https://imdb-classification.s3.ap-northeast-1.amazonaws.com/bert/pytorch_model.bin
INFO:networks.weights.load_weights_from_s3:complete read uri duraion in senconds: 928.8808689117432


### look at BertModel paramters 
check requierd_grad == **False**

In [10]:
for name, weight in net.named_parameters():
    print(name)
#     print(weight)
    print("-"*30)

bert.embedd.embeddings.weight
------------------------------
bert.embedd.position_embeddings.weight
------------------------------
bert.embedd.token_type_embeddings.weight
------------------------------
bert.embedd.norm.gamma
------------------------------
bert.embedd.norm.beta
------------------------------
bert.encode.layers.0.attnet.attnet.query.weight
------------------------------
bert.encode.layers.0.attnet.attnet.query.bias
------------------------------
bert.encode.layers.0.attnet.attnet.key.weight
------------------------------
bert.encode.layers.0.attnet.attnet.key.bias
------------------------------
bert.encode.layers.0.attnet.attnet.value.weight
------------------------------
bert.encode.layers.0.attnet.attnet.value.bias
------------------------------
bert.encode.layers.0.attnet.attout.dense.weight
------------------------------
bert.encode.layers.0.attnet.attout.dense.bias
------------------------------
bert.encode.layers.0.attnet.attout.LayerNorm.gamma
-------------------

In [14]:
from trainer import trainer 
import torch 

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=2e-5)

results_model = trainer(train, val, net, criterion, optimizer, 10, "training step 1 counts")

### testing step 

In [15]:
from evaluate import evaluate 

evaluate(test, net, criterion)

  0%|                                                                                                         | 0/313 [00:00<?, ?it/s]/home/kooose/my-lab/imdb-classification/work/models/bert/networks/bertmodel/attention_net.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weigths = torch.nn.functional.softmax(weights) #ここで<pad>の部分はほぼ0になる
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:53<00:00,  5.86it/s]

test loss: 0.082355 accuracy: 0.494200


### Attetion weights 

In [6]:
# import torch.onnx as onnx 

# onnx.export(net, (torch.rand(1, 2).long(), torch.rand(1, 2).long()), "sample2.onnx")

In [7]:
# net(torch.rand(1, 2).long(), torch.rand(1,2).long(), torch.rand(1, 2).long())

In [2]:
from pred.make_pkl import upload_s3_bucket

upload_s3_bucket("./pred/prediction/prediction_bert.pkl")

INFO:pred.make_pkl:upload file to s3 ... 
INFO:pred.make_pkl:complete upload files !!!


### saving prediction functions `.pkl`
* local 
* S3

In [ ]:
from pred.prediction import pred
from pred.preprocessing import prep
from pred.make_pkl import make_dump_prep, upload_s3_bucket 

make_dump_prep(prep)
filename_pred = "./pred/prediction/predicion.pkl"
make_dump_prep(pred, filename_pred)

upload_s3_bucket()
upload_s3_bucket(filename_pred)
